In [1]:
# notebook to convert a complete .npz file for a class to two 80:20 split TFRecords

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from numpy import asarray
from numpy import savez_compressed
import os
from numpy import load

In [3]:
# first lets get mapping done so that labels can be assigned

In [2]:
df = pd.read_csv('/content/drive/My Drive/train.csv')
ser = df['ebird_code']
lis = ser.tolist()
classes = np.unique(lis)
print(len(classes)) #264
dict_class = {}
dict_class = { classes[i] : i for i in range(0, len(classes))}
print(dict_class['amebit']) #2
print(dict_class['yerwar']) #262

264
2
262


In [3]:
# next, the functions for tf record creation

In [4]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [5]:
def serialize_example(feature0, feature1):
  feature = {
      'feature0': _bytes_feature(feature0),
      'feature1': _int64_feature(feature1),
  }
# # Create a Features message using tf.train.Example.
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [10]:
# splitting for every five seconds' worth of data

In [11]:
# output directories
train_output = '/content/drive/My Drive/lala1/Data-5Seconds/Train'
test_output = '/content/drive/My Drive/lala1/Data-5Seconds/Test'

In [13]:
# input directory
npz_dir = '/content/drive/My Drive/lala1/Keep Aside/output2-npz-full'
npz_files = os.listdir(npz_dir)

In [14]:
#loop through all classes
num = 0

for bird in npz_files:
    num = num+1
    label = dict_class[bird.split('.')[0]]

    bird_path = os.path.join(npz_dir, bird)
    dict_data = load(bird_path)
    data = dict_data['arr_0']
    data = np.transpose(data)


    output_file_train = train_output + '/' + bird.split('.')[0] + '_train.tfrecord'
    output_file_test = test_output + '/' + bird.split('.')[0] + '_test.tfrecord'

    print('class tf record for ', bird.split('.')[0], ' train is:', output_file_train)
    print('class tf record for ', bird.split('.')[0], ' test is:', output_file_test)

    options = tf.io.TFRecordOptions(compression_type="GZIP")

    test_array, train_array = data[0:int(0.2*len(data))], data[int(0.2*len(data)):]

    print(test_array.shape)
    print(train_array.shape)

    # loop through the array in the .npz file and add one by one to a tf record
    with tf.io.TFRecordWriter(output_file_train, options=options) as writer:
        for i in train_array:
            print('initial', i.shape)
            ii = np.array(np.split(i, 6))
            print('split and total', ii.shape)
            for iii in ii:
              print('each to be written', iii.shape)
              iii = tf.convert_to_tensor(iii)
              iii = tf.expand_dims(iii, -1)
              iii.set_shape([150000,1])

              serialized_example = serialize_example(tf.io.serialize_tensor(iii), label)
              writer.write(serialized_example)

    with tf.io.TFRecordWriter(output_file_test, options=options) as writer:

        for i in test_array:
            print('initial', i.shape)
            ii = np.array(np.split(i, 6))
            print('split and total', ii.shape)
            for iii in ii:
              print('each to be written', iii.shape)
              iii = tf.convert_to_tensor(iii)
              iii = tf.expand_dims(iii, -1)
              iii.set_shape([150000,1])

              serialized_example = serialize_example(tf.io.serialize_tensor(iii), label)
              writer.write(serialized_example)

Streaming output truncated to the last 5000 lines.
split and total (6, 150000)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
initial (900000,)
split and total (6, 150000)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
initial (900000,)
split and total (6, 150000)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
initial (900000,)
split and total (6, 150000)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
each to be written (150000,)
initial (900000,)
split and total (6, 150000)
each to be written (150000,)
each to be w